In [4]:
# Setup
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
import torch

# Steam Game Recommendation System (AI Prototype)
Goal : Build a minimal AI-based recommender using Matrix Factorization/Neural Collaborative Filtering

# Introduction
## Data Source
The dataset being used in this case is the Steam Video Game and Budle Data - User and Item data and item data from Professor Julian McAluley of the University of California San Diego

### Metadata and Key Features
#### Dataset 1 - user-item data
Size : 527 MB
Items : 88310
Data Range : 
Data Format : SteamID - Items{item_id, playtime_forever}
#### Dataset 2 - item data
Size :
Items :
Data Range : 
Data Format : ItemId - playtime_forever, tags, genres

In [ ]:
# load data (col: user_id, item_id, playtime, )
df = pd.read_csv()

# Exploratory Data Analysis

# Model Implementation

## Basline

## AI-Based Model

# Evaluation

# Demo

# Conclusion
## Summary
## Limitations
## Future